<a href="https://www.kaggle.com/code/quddusikashaf/projectbook?scriptVersionId=200275060" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Function to Load data from Multiple Datasets

In [49]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class MultiDatasetLoader(Dataset):
    def __init__(self, dataset_info_list, transform=None, num_samples=500):
        """
        Initializes the dataset loader for multiple datasets with varying configurations.

        :param dataset_info_list: A list where each item is a list containing:
                                  [image_dir, csv_file, image_column, label_column, binary (True/False)]
        :param transform: Transformations to be applied to the images.
        :param num_samples: Number of samples to load from each dataset.
        """
        self.transform = transform
        self.data = pd.DataFrame()  # Create an empty DataFrame to hold data from all datasets

        # Process each dataset in the dataset_info_list
        for dataset_info in dataset_info_list:
            image_dir, csv_file = dataset_info[:2]
            image_column = dataset_info[2] if len(dataset_info) > 2 else "name"
            label_column = dataset_info[3] if len(dataset_info) > 3 else "diagnosis"
            is_binary = dataset_info[4] if len(dataset_info) > 4 else True
            self.ext = dataset_info[5] if len(dataset_info) > 5 else ''

            # Load CSV file
#             data_part = pd.read_csv(csv_file)
            data_part = pd.read_csv(csv_file, dtype={image_column: str})

            # Convert labels to binary if needed
            if not is_binary:
                data_part['binary_label'] = data_part[label_column].apply(lambda x: 0 if x == 0 or x == 1 else 1)
                label_column = 'binary_label'  # Use the binary label for further processing
            else:
                data_part['binary_label'] = data_part[label_column]

            # Sample images from both classes equally
            class_0_count = len(data_part[data_part['binary_label'] == 0])
            class_1_count = len(data_part[data_part['binary_label'] == 1])
            min_samples_0 = min(class_0_count, num_samples)
            min_samples_1 = min(class_1_count, num_samples)
            print(f'Class 0: {min_samples_0}, Class 1: {min_samples_1}')
            # Sample data from each class
            data_0 = data_part[data_part['binary_label'] == 0].sample(n=min_samples_0, random_state=42)
            data_1 = data_part[data_part['binary_label'] == 1].sample(n=min_samples_1, random_state=42)

            # Concatenate and store dataset info
            data_part_sampled = pd.concat([data_0, data_1], axis=0)
            data_part_sampled['image_dir'] = image_dir  # Store the directory for the images

            # Append to the full dataset
            self.data = pd.concat([self.data, data_part_sampled], axis=0)

        # Shuffle the dataset
        self.data = self.data.sample(frac=1).reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image name, handle cases with or without extension
        img_name = self.data.iloc[idx]['name']
        image_dir = self.data.iloc[idx]['image_dir']

        # If the image name doesn't contain an extension, add one (e.g., '.png')
#         if not any(ext in img_name for ext in ['.png', '.jpg','jpeg']):
#             img_name = f"{img_name}.png"  # Default to .png, modify as per the image format you expect

        img_path = os.path.join(image_dir, img_name)

        try:
            # Open the image and convert it to RGB
            image = Image.open(img_path)
        except Exception as e:
            print(f"Error loading image: {img_path}. Skipping to next image.")
            return self.__getitem__((idx + 1) % len(self.data))  # Skip to next image if an error occurs

        # Get the label for the image
        label = self.data.iloc[idx]['binary_label']

        if self.transform:
            image = self.transform(image)

        return image, label


In [48]:
eyepacs_train=['/kaggle/input/eyepacs-ai4med/EyePACS/train','/kaggle/input/eyepacs-ai4med/EyePACS/train.csv']
eyepacs_test=['/kaggle/input/eyepacs-ai4med/EyePACS/test','/kaggle/input/eyepacs-ai4med/EyePACS/test.csv']
eyepacs_valid=['/kaggle/input/eyepacs-ai4med/EyePACS/valid','/kaggle/input/eyepacs-ai4med/EyePACS/valid.csv']

idrid_train=['/kaggle/input/idrid-ai4med/idrid/train','/kaggle/input/idrid-ai4med/idrid/train.csv']
idrid_test=['/kaggle/input/idrid-ai4med/idrid/test','/kaggle/input/idrid-ai4med/idrid/test.csv']
idrid_valid=['/kaggle/input/idrid-ai4med/idrid/valid','/kaggle/input/idrid-ai4med/idrid/valid.csv']

messidor_train=['/kaggle/input/messidor-ai4med/train','/kaggle/input/messidor-ai4med/train.csv']
messidor_test=['/kaggle/input/messidor-ai4med/test','/kaggle/input/messidor-ai4med/test.csv']
messidor_valid=['/kaggle/input/messidor-ai4med/valid','/kaggle/input/messidor-ai4med/valid.csv']

aptos_train=['/kaggle/input/aptos2019/train_images/train_images','/kaggle/input/aptos2019/train_1.csv','id_code','diagnosis',False,'.png']
aptos_test=['/kaggle/input/aptos2019/test_images/test_images','/kaggle/input/aptos2019/test.csv','id_code','diagnosis',False,'.png']
aptos_valid=['/kaggle/input/aptos2019/val_images/val_images','/kaggle/input/aptos2019/valid.csv','id_code','diagnosis',False,'.png']


hybrid_train=[eyepacs_train,idrid_train,messidor_train,aptos_train]
hybrid_test=[eyepacs_test,idrid_test,messidor_test,aptos_test]
hybrid_valid=[eyepacs_valid,idrid_valid,messidor_valid, aptos_valid]

## Train, Test and Validate Function

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
import numpy as np

# Training function
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Testing function
def test_model(model, dataloader, device):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    return accuracy_score(all_labels, all_preds)


## Inception V3

Early Stopping and Learning Scheduler has been used here. It has three different functions for Training, Validating and Testing.

In [12]:
from torchvision.models import Inception_V3_Weights  

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
import numpy as np


# Define image transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Parameters
batch_size = 32
learning_rate = 1e-4


# Prepare dataset
# train_image_dir = '/kaggle/input/hybrid-fundus-dataset/Train'
# train_csv_file = '/kaggle/input/hybrid-fundus-dataset/train.csv'
# train_dataset = BinaryClassificationDataset(image_dir=train_image_dir, csv_file=train_csv_file, transform=transform)

# test_image_dir = '/kaggle/input/hybrid-fundus-dataset/Test'
# test_csv_file = '/kaggle/input/hybrid-fundus-dataset/test.csv'
# test_dataset = BinaryClassificationDataset(image_dir=test_image_dir, csv_file=test_csv_file, transform=transform,num_samples=100,nameColumn="Name")

# valid_image_dir = '/kaggle/input/hybrid-fundus-dataset/Valid'
# valid_csv_file = '/kaggle/input/hybrid-fundus-dataset/valid.csv'
# valid_dataset = BinaryClassificationDataset(image_dir=valid_image_dir, csv_file=valid_csv_file, transform=transform,num_samples=100)


# Create dataset instance
train_dataset = MultiDatasetLoader(hybrid_train, transform=transform, num_samples=1000)
test_dataset = MultiDatasetLoader(hybrid_test, transform=transform, num_samples=1000)
valid_dataset = MultiDatasetLoader(hybrid_valid, transform=transform, num_samples=1000)


# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load pre-trained InceptionV3 model
model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
model.aux_logits = False
model.fc = nn.Linear(model.fc.in_features, 2)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)


1000
600
172
0
300
302
0
0
600
300
13
0


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:02<00:00, 37.9MB/s] 


cuda


In [13]:
num_epochs = 10
best_val_loss=np.inf
patience=3
patience_counter=0
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop with early stopping
for epoch in range(num_epochs):
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_model(model, val_loader, criterion, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# Test the model
test_acc = test_model(model, test_loader, device)
print(f"Test Accuracy: {test_acc:.4f}")

Validation: 100%|██████████| 58/58 [00:54<00:00,  1.06it/s]


Epoch 1/10
Train Loss: 0.3860, Train Accuracy: 0.8138
Validation Loss: 0.3007, Validation Accuracy: 0.8724


Validation: 100%|██████████| 58/58 [00:49<00:00,  1.16it/s]


Epoch 2/10
Train Loss: 0.2114, Train Accuracy: 0.9097
Validation Loss: 0.3149, Validation Accuracy: 0.8664


Validation: 100%|██████████| 58/58 [00:50<00:00,  1.16it/s]


...

Validation: 100%|██████████| 58/58 [00:48<00:00,  1.20it/s]


Epoch 5/10
Train Loss: 0.0693, Train Accuracy: 0.9760
Validation Loss: 0.3540, Validation Accuracy: 0.8773


Validation: 100%|██████████| 58/58 [00:49<00:00,  1.16it/s]

Epoch 6/10
Train Loss: 0.0521, Train Accuracy: 0.9817
Validation Loss: 0.3325, Validation Accuracy: 0.8965
Early stopping triggered


Test Accuracy: 0.9252


## ResNet 50

In [14]:
from torchvision.models import ResNet50_Weights
from torchvision.transforms import InterpolationMode

# Define data transformations using ResNet18 inference transforms
restnet_preprocess = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),  # Resizing to 256
    transforms.CenterCrop(224),                                        # Central crop of 224
    transforms.ToTensor(),                                             
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])
batch_size=32
# Prepare datasets
# resnet_train_dataset = BinaryClassificationDataset(image_dir=train_image_dir, csv_file=train_csv_file, transform=restnet_preprocess)
# resnet_test_dataset = BinaryClassificationDataset(image_dir=test_image_dir, csv_file=test_csv_file, transform=restnet_preprocess, num_samples=300,nameColumn="Name")
# resnet_valid_dataset = BinaryClassificationDataset(image_dir=valid_image_dir, csv_file=valid_csv_file, transform=restnet_preprocess, num_samples=300)
resnet_train_dataset = MultiDatasetLoader(hybrid_train, transform=restnet_preprocess, num_samples=1000)
resnet_test_dataset = MultiDatasetLoader(hybrid_test, transform=restnet_preprocess, num_samples=1000)
resnet_valid_dataset = MultiDatasetLoader(hybrid_valid, transform=restnet_preprocess, num_samples=1000)



# Data loaders
resnet_train_loader = DataLoader(resnet_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
resnet_val_loader = DataLoader(resnet_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
resnet_test_loader = DataLoader(resnet_test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


resnet_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)  # Adjust the final layer for binary classification

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet_model.to(device)
print('')

1000
600
172
0
300
302
0
0
600
300
13
0


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s] 


In [15]:

# Define loss, optimizer, and learning rate scheduler
resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
resnet_scheduler = optim.lr_scheduler.StepLR(resnet_optimizer, step_size=5, gamma=0.1)

# Early stopping setup
best_val_loss = float('inf')
patience_counter = 0
patience=3
num_epochs=10
# Training loop with early stopping
for epoch in range(num_epochs):
    train_loss, train_acc = train_model(resnet_model, resnet_train_loader, resnet_criterion, resnet_optimizer, device)
    val_loss, val_acc = validate_model(resnet_model, resnet_val_loader, resnet_criterion, device)
    resnet_scheduler.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# Test the model
test_acc = test_model(resnet_model, resnet_test_loader, device)
print(f"Test Accuracy: {test_acc:.4f}")

Validation: 100%|██████████| 58/58 [00:37<00:00,  1.56it/s]


Epoch 1/10
Train Loss: 0.4847, Train Accuracy: 0.7689
Validation Loss: 0.4475, Validation Accuracy: 0.7886


Validation: 100%|██████████| 58/58 [00:37<00:00,  1.55it/s]


Epoch 2/10
Train Loss: 0.3495, Train Accuracy: 0.8496
Validation Loss: 0.4110, Validation Accuracy: 0.8182


Validation: 100%|██████████| 58/58 [00:36<00:00,  1.59it/s]


...

Validation: 100%|██████████| 58/58 [00:36<00:00,  1.59it/s]


Epoch 8/10
Train Loss: 0.0578, Train Accuracy: 0.9834
Validation Loss: 0.3582, Validation Accuracy: 0.8724


Validation: 100%|██████████| 58/58 [00:36<00:00,  1.59it/s]

Epoch 9/10
Train Loss: 0.0318, Train Accuracy: 0.9924
Validation Loss: 0.3983, Validation Accuracy: 0.8740
Early stopping triggered


Test Accuracy: 0.9103


## EfficientNet-B4

In [50]:
import torch
# import torch_xla.core.xla_model as xm
from torchvision.models import EfficientNet_B4_Weights
from torchvision.transforms import InterpolationMode
# Define EfficientNet-B7 inference preprocessing transforms
efficientnet_preprocess = transforms.Compose([
    transforms.Resize(384, interpolation=InterpolationMode.BICUBIC),  # Resize to 600 using BICUBIC interpolation
    transforms.CenterCrop(380),                                        # Central crop of 600
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])
# train_image_dir = '/kaggle/input/aptos2019/train_images/train_images'
# train_csv_file = '/kaggle/input/aptos2019/train_1.csv'
# test_image_dir = '/kaggle/input/aptos2019/test_images/test_images'
# test_csv_file = '/kaggle/input/aptos2019/test.csv'
# valid_image_dir = '/kaggle/input/aptos2019/val_images/val_images'
# valid_csv_file = '/kaggle/input/aptos2019/valid.csv'


# Prepare datasets
# efficientnet_train_dataset = BinaryClassificationDataset(image_dir=train_image_dir, csv_file=train_csv_file, transform=efficientnet_preprocess,num_samples=550)
# efficientnet_valid_dataset = BinaryClassificationDataset(image_dir=valid_image_dir, csv_file=valid_csv_file, transform=efficientnet_preprocess, num_samples=200)
# efficientnet_test_dataset = BinaryClassificationDataset(image_dir=test_image_dir, csv_file=test_csv_file, transform=efficientnet_preprocess, num_samples=200,nameColumn="Name")
efficientnet_train_dataset = MultiDatasetLoader(hybrid_train, transform = efficientnet_preprocess, num_samples=1000)
efficientnet_test_dataset = MultiDatasetLoader(hybrid_test, transform = efficientnet_preprocess, num_samples=1000)
efficientnet_valid_dataset = MultiDatasetLoader(hybrid_valid, transform = efficientnet_preprocess, num_samples=1000)


# Data loaders
efficientnet_train_loader = DataLoader(efficientnet_train_dataset, batch_size=16, shuffle=True, num_workers=4)
efficientnet_val_loader = DataLoader(efficientnet_valid_dataset, batch_size=16, shuffle=False, num_workers=4)
efficientnet_test_loader = DataLoader(efficientnet_test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Initialize the EfficientNet-B7 model

efficientnet_model = models.efficientnet_b4(weights=EfficientNet_B4_Weights.IMAGENET1K_V1)
efficientnet_model.classifier[1] = nn.Linear(efficientnet_model.classifier[1].in_features, 2)  # Adjust final layer for binary classification
efficientnet_model.to(device)

# Define the loss, optimizer, and learning rate scheduler
efficientnet_criterion = nn.CrossEntropyLoss()
efficientnet_optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)
efficientnet_scheduler = optim.lr_scheduler.CosineAnnealingLR(efficientnet_optimizer, T_max=10)


Class 0: 1000, Class 1: 1000
Class 0: 600, Class 1: 600
Class 0: 172, Class 1: 361
Class 0: 1000, Class 1: 1000
Class 0: 300, Class 1: 300
Class 0: 305, Class 1: 302
Class 0: 0, Class 1: 203
Class 0: 229, Class 1: 137
Class 0: 600, Class 1: 600
Class 0: 300, Class 1: 300
Class 0: 13, Class 1: 250
Class 0: 212, Class 1: 154


In [ ]:

# Early stopping setup
best_val_loss_efficientnet = float('inf')
patience=3
# Training loop for EfficientNet-B7 with early stopping
num_epoch=10
patience_counter_efficientnet=0
print("Training EfficientNet-B7...")
for epoch in range(num_epoch):
    train_loss_efficientnet, train_acc_efficientnet = train_model(efficientnet_model, efficientnet_train_loader, efficientnet_criterion, efficientnet_optimizer, device)
    val_loss_efficientnet, val_acc_efficientnet = validate_model(efficientnet_model, efficientnet_val_loader, efficientnet_criterion, device)
    efficientnet_scheduler.step()

    print(f"EfficientNet-B7 Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss_efficientnet:.4f}, Train Accuracy: {train_acc_efficientnet:.4f}")
    print(f"Validation Loss: {val_loss_efficientnet:.4f}, Validation Accuracy: {val_acc_efficientnet:.4f}")

    # Early stopping check
    if val_loss_efficientnet < best_val_loss_efficientnet:
        best_val_loss_efficientnet = val_loss_efficientnet
        patience_counter_efficientnet = 0
    else:
        patience_counter_efficientnet += 1
        if patience_counter_efficientnet >= patience:
            print("Early stopping for EfficientNet-B7 triggered")
            break

# Test the EfficientNet-B7 model
test_acc_efficientnet = test_model(efficientnet_model, efficientnet_test_loader, device)
print(f"EfficientNet-B4 Test Accuracy: {test_acc_efficientnet:.4f}")

## ViT

In [34]:
from torchvision.models import ViT_B_32_Weights
from torchvision import models, transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# Define data transformations using Vision Transformer inference transforms
vit_preprocess = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),  # Resize to 256
    transforms.CenterCrop(224),                                        # Central crop of 224
    transforms.ToTensor(),                                             
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])
batch_size=32

# Prepare datasets
# vit_train_dataset = BinaryClassificationDataset(image_dir=train_image_dir, csv_file=train_csv_file, transform=vit_preprocess)
# vit_test_dataset = BinaryClassificationDataset(image_dir=test_image_dir, csv_file=test_csv_file, transform=vit_preprocess, num_samples=100,nameColumn="Name")
# vit_valid_dataset = BinaryClassificationDataset(image_dir=valid_image_dir, csv_file=valid_csv_file, transform=vit_preprocess, num_samples=100)
vit_train_dataset = MultiDatasetLoader(hybrid_train, transform = vit_preprocess, num_samples=1000)
vit_test_dataset = MultiDatasetLoader(hybrid_test, transform = vit_preprocess, num_samples=1000)
vit_valid_dataset = MultiDatasetLoader(hybrid_valid, transform = vit_preprocess, num_samples=1000)

# Data loaders
vit_train_loader = DataLoader(vit_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
vit_val_loader = DataLoader(vit_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
vit_test_loader = DataLoader(vit_test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load pre-trained Vision Transformer (ViT-B/16) model
vit_model = models.vit_b_32(weights=ViT_B_32_Weights.IMAGENET1K_V1)
vit_model.heads.head = nn.Linear(vit_model.heads.head.in_features, 2)  # Adjust the final layer for binary classification


Class 0: 1000, Class 1: 1000
Class 0: 600, Class 1: 600
Class 0: 172, Class 1: 361
Class 0: 300, Class 1: 300
Class 0: 305, Class 1: 302
Class 0: 0, Class 1: 203
Class 0: 229, Class 1: 137
Class 0: 600, Class 1: 600
Class 0: 300, Class 1: 300
Class 0: 13, Class 1: 250
Class 0: 212, Class 1: 154


In [ ]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model.to(device)

# Define loss, optimizer, and learning rate scheduler
vit_criterion = nn.CrossEntropyLoss()
vit_optimizer = optim.AdamW(vit_model.parameters(), lr=0.00005)
vit_scheduler = optim.lr_scheduler.StepLR(vit_optimizer, step_size=5, gamma=0.1)

# Early stopping setup
best_val_loss = float('inf')
patience_counter = 0
patience = 3
num_epochs = 20

# Training loop with early stopping
for epoch in range(num_epochs):
    train_loss, train_acc = train_model(vit_model, vit_train_loader, vit_criterion, vit_optimizer, device)
    val_loss, val_acc = validate_model(vit_model, vit_val_loader, vit_criterion, device)
    vit_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

#     # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# Test the model
test_acc = test_model(vit_model, vit_test_loader, device)
print(f"Test Accuracy: {test_acc:.4f}")


# Appendix


In [ ]:
# Parameters
image_dir = '/kaggle/input/hybrid-fundus-dataset/Test'
csv_file = '/kaggle/input/hybrid-fundus-dataset/test.csv'
n = 500  # Number of images per class
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset and dataloader
dataset = BinaryClassificationDataset(image_dir=image_dir, csv_file=csv_file, transform=transform,nameColumn="Name")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# Example usage: Iterating through the DataLoader
for images, labels in dataloader:
    print(images.shape, labels.shape)
    break